Importing the NonPredictive class from the nonpredictive module:

In [2]:
import sys
sys.path.append('..')
from glsm.non_predictive import NonPredictive
from glsm.features import Feature

Instanciate the model and add the features:

In [3]:
model = NonPredictive()

feature_a = Feature(
    name="Monthly Users",
    weight=0.5,
    points_map=[
        ("Up to 50K",00),
        ("50K - 100K",50),
        ("100K - 200K",70),
        ("More than 200K",100),
    ]
)
model.add_features([feature_a])
feature_b = Feature(
    name="Industry",
    weight=0.25,
    points_map=[
        ("Technology",70),
        ("Real State",20),
        ("Retail",50),
        ("Education",50),
        ("Health",100),
    ]
)
model.add_features([feature_b])
feature_c = Feature(
    name="Mkt Investment",
    weight=1,
    points_map=[
        ("Up to $50K",0),
        ("50k - $100K",30),
        ("100k - $200K",50),
        ("$200K - $300K",70),
        ("$300K - $400K",90),
        ("More than $400K",100),
    ]
)
model.add_features([feature_c])

Read leads from a csv file and add lambda values to a new csv file:

In [7]:
import csv # or you can just use pandas instead

with open('leads.csv', 'r') as file:
    if file.read(3) == b'\xef\xbb\xbf':
        file.seek(3)
    csv_reader = csv.reader(file)
    headers = next(csv_reader)

    # New csv file adding lambda values
    with open('leads_with_lambda.csv', 'w', newline='') as new_file:
        csv_writer = csv.writer(new_file)
        csv_writer.writerow(headers + ['lambda'])

        for row in csv_reader:
            lead = dict(zip(headers, row))
            lambda_value = model.compute_lambda(lead)
            new_row = row + [lambda_value]
            print(new_row)
            csv_writer.writerow(new_row)


['50K - 100K', 'Technology', '$300K - $400K', 81.43]
['50K - 100K', 'Retail', '100k - $200K', 50.0]
['Up to 50K', 'Real State', '50k - $100K', 23.81]


Compute lambda values for a single lead:

In [5]:
lambda_value = model.compute_lambda(lead)
print(lambda_value)

23.81
